# AlexNet implementation

In [41]:
import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm

In [35]:
class AlexNet(nn.Module):
  def __init__(self, in_channels=3, classes=10): #3 input channels because we use RGB, and we'll have 1000 output classes (this is given by the dataset we're using)
    super().__init__()
    # Initialize each layer
    self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=96, kernel_size=11, stride=4, padding=0)
    self.c2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
    self.c3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
    self.c4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
    self.c5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
    self.fc1 = nn.Linear(in_features=6*6*256, out_features=4096)
    self.fc2 = nn.Linear(in_features=4096, out_features=4096)
    self.fc3 = nn.Linear(in_features=4096, out_features=classes)
    self.norm = nn.LocalResponseNorm(k=2, size=5, alpha=1e-4, beta=0.75)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.5)

    # Initialize weights
    self.weight_init()

  def forward(self, x):
      x = self.maxpool(self.norm(self.relu(self.c1(x))))
      x = self.maxpool(self.norm(self.relu(self.c2(x))))
      x = self.relu(self.c3(x))
      x = self.relu(self.c4(x))
      x = self.maxpool(self.relu(self.c5(x)))
      x = torch.flatten(x,1)
      x = self.relu(self.fc1(self.dropout(x)))
      x = self.relu(self.fc2(self.dropout(x)))
      x = self.fc3(x)
      return x

  def weight_init(self):
    bias = [1,3,4,5,6,7]
    for i, layer in enumerate(self.modules()):
      if layer is nn.Conv2d or layer is nn.Linear:
        nn.init.normal_(mean=0, std=0.01)
        if i in bias:
          nn.init.constant_(layer, 1)
        else:
          nn.init.constant_(layer, 0)



### Training

In [47]:
# Load CIFAR-10 train dataset
data_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
cifar10_train_dataset = torchvision.datasets.CIFAR10(root='./data', download=True, train=True, transform=data_transforms)
train_dataloader = torch.utils.data.DataLoader(cifar10_train_dataset,
                                         batch_size = 32,
                                         shuffle=True)

# Load AlexNet
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG16_A().to(device)
model.train()

# Define Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

epoch_nums = 2 # small just for working demonstration
# Train the network
for epoch in range(epoch_nums):

  current_loss = 0.0
  for i, data in tqdm(enumerate(train_dataloader, start = 0), unit="batch", total=len(train_dataloader), desc=f"Epoch {epoch}"):
    # get inputs -> data is a list of [inputs, labels]
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    # reset gradients
    optimizer.zero_grad()

    # forward pass + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # compute statistics
    current_loss += loss.item()
    if i % 100 == 99:    # print every 100 mini-batches
            print(f'current loss: {current_loss / 100:.3f}')
            current_loss = 0.0

print("Training done! Saving trained model to: './trained_model.pth'")
torch.save(model.state_dict(), './trained_model.pth')
print("Saved.")






Files already downloaded and verified


Epoch 0:   6%|▋         | 101/1563 [00:10<02:58,  8.19batch/s]

current loss: 2.303


Epoch 0:  13%|█▎        | 201/1563 [00:21<02:24,  9.43batch/s]

current loss: 2.300


...

current loss: 0.958


Epoch 1:  96%|█████████▌| 1501/1563 [02:44<00:07,  8.86batch/s]

current loss: 0.971


Epoch 1: 100%|██████████| 1563/1563 [02:50<00:00,  9.14batch/s]


Training done! Saving trained model to: './trained_model'
Saved.


### Testing

In [50]:
# Load CIFAR-10 test dataset
data_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((227,227)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
cifar10_test_dataset = torchvision.datasets.CIFAR10(root='./data', download=True, train=False, transform=data_transforms)
test_dataloader = torch.utils.data.DataLoader(cifar10_test_dataset,
                                         batch_size = 32,
                                         shuffle=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AlexNet().to(device)
model.load_state_dict(torch.load("./trained_model.pth"))

correct = 0
total = 0
with torch.no_grad():
  for data in tqdm(test_dataloader):
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'\nAccuracy of AlexNet on CIFAR10: {100 * correct // total} %')

Files already downloaded and verified


100%|██████████| 313/313 [00:24<00:00, 12.67it/s]


Accuracy of AlexNet on CIFAR10: 68 %
